# 導入環境

In [ ]:
!pip install datasets
!pip install accelerate -U
!pip install transformers[torch]
!pip install peft
!pip install pandas pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [ ]:
!apt-get install wget
!wget https://huggingface.co/datasets/ChenWeiLi/Medtext_zhtw/raw/main/MedText_zhtw.json

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
wget is already the newest version (1.21.2-2ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
--2024-07-13 11:44:52--  https://huggingface.co/datasets/ChenWeiLi/Medtext_zhtw/raw/main/MedText_zhtw.json
Resolving huggingface.co (huggingface.co)... 18.172.134.124, 18.172.134.88, 18.172.134.24, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 983900 (961K) [text/plain]
Saving to: ‘MedText_zhtw.json’

MedText_zhtw.json   100%[===================>] 960.84K  --.-KB/s    in 0.08s   

2024-07-13 11:44:53 (12.3 MB/s) - ‘MedText_zhtw.json’ saved [983900/983900]



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers datasets
import os
from huggingface_hub import login
from google.colab import output
# 替換為你的Hugging Face API Token
#os.environ["HUGGINGFACE_TOKEN"] = "your HUGGINGFACE_TOKEN"
os.environ["HUGGINGFACE_TOKEN"] = "your HUGGINGFACE_TOKEN"
# 登錄Hugging Face
login(token=os.environ["HUGGINGFACE_TOKEN"])
#output.eval_js('''google.colab.secrets.set("HF_TOKEN", "your HUGGINGFACE_TOKEN")''')

#login("your HUGGINGFACE_TOKEN")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig

In [ ]:
# 將 JSON 文件轉換為 CSV 文件。
df = pd.read_json('/content/MedText_zhtw.json' )
ds = Dataset.from_pandas(df)

In [ ]:
ds[:3]

{'instruction': ['你是一位專業的醫療人員,請用心且專業的回答問題。',
  '你是一位專業的醫療人員,請用心且專業的回答問題。',
  '你是一位專業的醫療人員,請用心且專業的回答問題。'],
 'input': ['一名 50 歲男性有復發性腎結石和骨質減少病史。由於先前診斷出維生素 D 缺乏症，他一直在服用大劑量的維生素 D 補充劑。實驗室結果顯示高血鈣症和高鈣尿症。可能的診斷是什麼，治療方法是什麼？',
  '一名 7 歲男孩出現發燒、頭痛和嚴重耳痛。他還抱怨頭暈和旋轉感。檢查發現鼓膜呈紅色、鼓起。鑑別診斷有哪些，下一步該做什麼？',
  '一名 35 歲女性出現持續乾咳、氣短和疲勞。她最初被懷疑患有氣喘，但使用支氣管擴張劑後，她的肺量測定結果並未改善。診斷結果可能是什麼？'],
 'output': ['該患者有復發性腎結石、骨質減少和大劑量維生素 D 補充劑病史，以及高鈣血症和高鈣尿症的實驗室檢查結果，暗示維生素 D 中毒的可能性。過量攝取維生素 D 會造成腸道對鈣的吸收增加，導致高鈣血症和高鈣尿症，及腎結石和骨質流失。治療包括停止補充維生素 D，並可能提供靜脈輸液和袢利尿劑以促進鈣的排泄。',
  '該孩子的鼓膜發紅、鼓起，伴隨發燒和頭痛等全身症狀，以及頭暈和旋轉感等其他症狀，引起人們對急性中耳炎併發症的關注。鑑別診斷可能包括迷路炎甚至乳突炎。可能需要緊急評估，包括進一步的影像學研究，例如 CT 或 MRI 掃描。如果確診乳突炎，該兒童可能需要接受靜脈注射抗生素治療，也可能需要進行手術介入。',
  '雖然這些症狀最初可能暗示患有氣喘，但對支氣管擴張劑缺乏反應表明有不同的原因。在這種情況下，可能的診斷可能是特發性肺纖維化，這是一種肺部疾病，會因未知原因導致肺部疤痕（纖維化）。胸部高解析度 CT 將是診斷的下一步。'],
 'history': [[], [], []]}

# 处理数据集

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
tokenizer = AutoTokenizer.from_pretrained('taide/TAIDE-LX-7B', use_fast=False, trust_remote_code=True)
#tokenizer = AutoTokenizer.from_pretrained('taide/Llama3-TAIDE-LX-8B-Chat-Alpha1', use_fast=False, trust_remote_code=True)

tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/813k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

In [ ]:
def process_func(example):
    MAX_LENGTH = 384  # Llama 分詞器會將一個中文字切割成多個 token，因此需要放開一些最大長度，確保資料的完整性
    input_ids, attention_mask, labels = [], [], []

    # 構建 instruction 和 input 字符串，並進行分詞
    instruction = tokenizer(f"user\n\n{example['instruction'] + example['input']}assistant\n\n", add_special_tokens=False)
    response = tokenizer(f"{example['output']}", add_special_tokens=False)

    # 合併 input 和 response 的 token ID 和注意力掩碼
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]

    # 如果超過最大長度，進行截斷
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]

    # 確保 attention_mask 和 labels 與 input_ids 一致
    attention_mask = attention_mask[:len(input_ids)]
    labels = labels[:len(input_ids)]

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

# 應用到整個數據集
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)

print(tokenized_ds)
print(tokenizer.decode(tokenized_ds[0]['input_ids']))
print(tokenizer.decode([token for token in tokenized_ds[0]["labels"] if token != -100]))


Map:   0%|          | 0/1412 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1412
})
user

你是一位專業的醫療人員,請用心且專業的回答問題。一名 50 歲男性有復發性腎結石和骨質減少病史。由於先前診斷出維生素 D 缺乏症，他一直在服用大劑量的維生素 D 補充劑。實驗室結果顯示高血鈣症和高鈣尿症。可能的診斷是什麼，治療方法是什麼？assistant

 該患者有復發性腎結石、骨質減少和大劑量維生素 D 補充劑病史，以及高鈣血症和高鈣尿症的實驗室檢查結果，暗示維生素 D 中毒的可能性。過量攝取維生素 D 會造成腸道對鈣的吸收增加，導致高鈣血症和高鈣尿症，及腎結石和骨質流失。治療包括停止補充維生素 D，並可能提供靜脈輸液和袢利尿劑以促進鈣的排泄。
該患者有復發性腎結石、骨質減少和大劑量維生素 D 補充劑病史，以及高鈣血症和高鈣尿症的實驗室檢查結果，暗示維生素 D 中毒的可能性。過量攝取維生素 D 會造成腸道對鈣的吸收增加，導致高鈣血症和高鈣尿症，及腎結石和骨質流失。治療包括停止補充維生素 D，並可能提供靜脈輸液和袢利尿劑以促進鈣的排泄。


# 創建模型

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification
#model = AutoModelForCausalLM.from_pretrained('meta-llama/Meta-Llama-3-8B-Instruct', device_map="auto",torch_dtype=torch.bfloat16)
#model = AutoModelForCausalLM.from_pretrained('/root/autodl-tmp/LLM-Research/Meta-Llama-3-8B-Instruct', device_map="auto",torch_dtype=torch.bfloat16)
#model = AutoModelForCausalLM.from_pretrained("taide/TAIDE-LX-7B", device_map="auto",torch_dtype=torch.bfloat16)#先前用的
model = AutoModelForCausalLM.from_pretrained("taide/TAIDE-LX-7B", device_map="auto",torch_dtype=torch.bfloat16)
# 從檢查點載入模型
#model_checkpoint = '/content/drive/MyDrive/colab_results/checkpoint-38500'  # 修改為最新的檢查點路徑
#model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
# 重新開始加載模型
#model = AutoModelForCausalLM.from_pretrained('taide/Llama3-TAIDE-LX-8B-Chat-Alpha1', device_map="auto",torch_dtype=torch.float32)

# 設定 pad_token_id
model.config.pad_token_id = tokenizer.pad_token_id
# 設定 eos_token_id
model.config.eos_token_id = tokenizer.eos_token_id

model

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(56064, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head):

In [ ]:
model.enable_input_require_grads() # 開啟梯度檢查點時，要執行方法

In [ ]:
model.dtype

torch.bfloat16

# lora

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, # 訓練模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alapa，具體作用參見 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'k_proj', 'o_proj', 'down_proj', 'gate_proj', 'q_proj', 'v_proj', 'up_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [ ]:
model = get_peft_model(model, config)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='taide/TAIDE-LX-7B', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'k_proj', 'o_proj', 'down_proj', 'gate_proj', 'q_proj', 'v_proj', 'up_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [ ]:
model.print_trainable_parameters()

trainable params: 19,988,480 || all params: 6,955,536,384 || trainable%: 0.2874


# 配置训练参数

In [ ]:
from transformers import TrainingArguments, AdamW, Trainer, get_linear_schedule_with_warmup

torch.utils.checkpoint.use_reentrant = False

# 設定訓練參數
args = TrainingArguments(
    output_dir="/content/drive/MyDrive/colab_results",
    save_steps=50,  # 每50步保存一次檢查點
    logging_dir='/content/drive/MyDrive/colab_logs',
    per_device_train_batch_size=64,  # 增加批次大小
    gradient_accumulation_steps=4,  # 調整累積梯度步數，使得實際批次大小達到 260K tokens
    logging_steps=10,
    num_train_epochs=40,
    learning_rate=5e-5,
    save_strategy="steps",  # 每 steps 次保存一次
    save_total_limit=5,  # 保留最多5個檢查點
    gradient_checkpointing=True,
    weight_decay=0.01,
)
# 設定優化器
#optimizer = AdamW
# 設定優化器
optimizer = AdamW(model.parameters(), lr=args.learning_rate, weight_decay=args.weight_decay)

# 設定學習率調度器
num_training_steps = len(tokenized_ds) // (args.per_device_train_batch_size * args.gradient_accumulation_steps) * args.num_train_epochs
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(0.1 * num_training_steps),  # 預熱步數，這裡設定為總步數的10%
    num_training_steps=num_training_steps
)

# 創建 Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds,
    optimizers=(optimizer, lr_scheduler),  # 設定優化器和學習率調度器
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
    tokenizer =tokenizer
)



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
#trainer = Trainer(
#    model=model,
#    args=args,
#    train_dataset=tokenized_ds,
#    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
#    tokenizer =tokenizer
#)

In [ ]:
trainer.train()

# 繼續訓練
#def forward_with_checkpoint(*args, **kwargs):
#    return torch.utils.checkpoint.checkpoint(trainer.training_step, *args, use_reentrant=False, **kwargs)

#trainer.training_step = forward_with_checkpoint
#trainer.train(resume_from_checkpoint=True)

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,2.267300
20,2.146600
30,1.968100
40,1.855300
50,1.768800
60,1.709000
70,1.633700
80,1.552100
90,1.487500
100,1.413200


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_downlo

TrainOutput(global_step=200, training_loss=1.465201473236084, metrics={'train_runtime': 3406.6581, 'train_samples_per_second': 16.579, 'train_steps_per_second': 0.059, 'total_flos': 4.305480986610893e+17, 'train_loss': 1.465201473236084, 'epoch': 34.78260869565217})

# 儲存 LoRA 和 tokenizer 結果


In [ ]:
peft_model_id="./llama3_lora"
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('./llama3_lora/tokenizer_config.json',
 './llama3_lora/special_tokens_map.json',
 './llama3_lora/tokenizer.model',
 './llama3_lora/added_tokens.json')

# 載入 lora 權重推理

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel
import gc
gc.collect()
#model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
#mode_path = 'meta-llama/Meta-Llama-3-8B-Instruct'
model_path = "taide/TAIDE-LX-7B"
lora_path = './llama3_lora'

# 加载tokenizer
#tokenizer = tokenizer
#tokenizer = AutoTokenizer.from_pretrained(mode_path)
#tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
tokenizer = AutoTokenizer.from_pretrained('taide/TAIDE-LX-7B', use_fast=False, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token


tokenizer.add_special_tokens({'pad_token': '[PAD]', 'bos_token': '[BOS]', 'eos_token': '[EOS]'})


# 加载模型
#model = AutoModelForCausalLM.from_pretrained(mode_path, device_map="auto",torch_dtype=torch.bfloat16).to('cuda')
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="cuda",torch_dtype=torch.bfloat16).to('cuda') # Load the model directly onto the GPU


#model = model
# 加载lora权重
model = PeftModel.from_pretrained(model, model_id=lora_path, config=config).to('cuda')

prompt = "每天只睡三小時會有啥狀況？"
messages = [
    # {"role": "system", "content": "现在你要扮演皇帝身边的女人--甄嬛"},
    {"role": "system", "content": "你是一位專業的醫療人員,請用心且專業的回答問題。"},
    {"role": "user", "content": prompt}
]

# 將消息合併成一個文本輸入
def format_messages(messages):
    formatted_messages = ""
    for message in messages:
        if message['role'] == 'system':
            formatted_messages += f"[SYSTEM] {message['content']}\n"
        elif message['role'] == 'user':
            formatted_messages += f"[USER] {message['content']}\n"
    return formatted_messages

# 格式化消息
formatted_text = format_messages(messages)
print("Formatted text:", formatted_text)

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
print("Generated text with chat template:", text)


model_inputs = tokenizer([text], return_tensors="pt").to('cuda')
# 打印model_inputs進行檢查
print("Model inputs:", model_inputs)

# 确认eos_token_id
eos_token_id = tokenizer.encode('</s>')[0]
print(f"EOS Token ID: {eos_token_id}")


#pad_token_id = tokenizer.pad_token_id
generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=384,
    #eos_token_id=tokenizer.encode('<|eot_id|>')[0],
    eos_token_id=eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
    attention_mask=model_inputs.attention_mask,
    repetition_penalty=1.6,  # 增加 repetition_penalty
    top_k=50,  # 設定最高k個概率詞
   # stop_token=tokenizer.eos_token  # 确保模型在遇到结束标记时停止
    do_sample=True,
    top_p=0.15,  # 新增 top_p 參數
    temperature=0.15,  # 設定 temperature 參數
    forced_eos_token_id=tokenizer.encode('</s>')[0]
)

#generated_ids = [
#    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
#]
# 去掉輸入部分，僅保留生成的文本
generated_ids = generated_ids[:, model_inputs.input_ids.shape[-1]:]


response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)


# 清理生成的文本
import re
response = re.sub(r'</s>.*', '', response, flags=re.DOTALL)
response = re.sub(r'</s>.*', '</s>', response)
response = re.sub(r'\[.*?\]', '', response)  # 去除方括號內的內容
response = re.sub(r'</?[^>]+>', '', response)  # 去除HTML標籤
response = re.sub(r'dress|dressing', '', response, flags=re.IGNORECASE)  # 去除 "dress" 和 "dressing"
response = re.sub(r'<<.*?>>', '', response)  # 去除 `<<SYS>>` 標籤
response = re.sub(r'\[.*?\]', '', response)  # 去除 `[/EMBB]]` 標籤
response = response.strip()  # 去除前后多余的空格
# 从最后开始查找句号并去除句号之后的语句
def remove_after_last_period(text):
    last_period_index = max(text.rfind('。'), text.rfind('!'))
    if last_period_index != -1:
        return text[:last_period_index + 1]
    return text

# 清理生成的文本
response = remove_after_last_period(response).strip()

print(response)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

No chat template is set for this tokenizer, falling back to a default class-level template. This is very error-prone, because models are often trained with templates different from the class default! Default chat templates are a legacy feature and will be removed in Transformers v4.43, at which point any code depending on them will stop working. We recommend setting a valid chat template before then to ensure that this model continues working without issues.


Formatted text: [SYSTEM] 你是一位專業的醫療人員,請用心且專業的回答問題。
[USER] 每天只睡三小時會有啥狀況？

Generated text with chat template: [BOS][INST] <<SYS>>
你是一位專業的醫療人員,請用心且專業的回答問題。
<</SYS>>

每天只睡三小時會有啥狀況？ [/INST]
Model inputs: {'input_ids': tensor([[56021, 56021, 29961, 25580, 29962,  3532, 14816, 29903,  6778,    13,
         33013, 32052, 37319, 52781, 32701, 29892, 50772, 44775, 32350, 37319,
         35616, 35211, 30267,    13, 29966,   829, 14816, 29903,  6778,    13,
            13, 42265, 31557, 45710, 30457, 37374, 40939, 35236, 44124, 30882,
           518, 29914, 25580, 29962]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}
EOS Token ID: 56021
如果一個人在很長一段時間內睡眠不足，可能會出現認知和精神症狀（例如注意力不集中、記憶力喪失或憂鬱）以及肌肉緊張甚至疼痛等身體變化跡象。長期的這種情況可能導致嚴重的健康後果並增加患慢性疲勞症候群的風險。因此應將此人轉介給醫生進行評估以排除其他潛在原因並提供適當的治療建議。也應該教育患者了解健康的生活方式如何有助於改善他們的整體身心狀態。提供支持性護理也很重要：確保他們保持良好的營養攝取量並且避免飲酒

In [ ]:
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.7 MB/s eta 0:00:00


In [ ]:
!mkdir ./model

In [ ]:
!pwd

/content


In [ ]:
from accelerate import Accelerator
accelerator = Accelerator()

# 保存模型
accelerator.save(model.state_dict(), './model/model.pth')

# 保存tokenizer
tokenizer.save_pretrained('./model')

In [ ]:
!ls -l

total 980
drwx------ 6 root root   4096 Jul 13 11:45 drive
drwxr-xr-x 2 root root   4096 Jul 13 12:44 llama3_lora
-rw-r--r-- 1 root root 983900 Jul 13 11:44 MedText_zhtw.json
drwxr-xr-x 2 root root   4096 Jul 13 12:45 model
drwxr-xr-x 1 root root   4096 Jul 11 13:22 sample_data


In [ ]:
import shutil
shutil.copytree('/content/model/', '/content/drive/MyDrive/test_G_save/model/')
#shutil.copy('/content/model/model.pth', '/content/drive/MyDrive/test_G_save/model/')

FileExistsError: [Errno 17] File exists: '/content/drive/MyDrive/test_G_save/model/'

In [ ]:
shutil.copytree('/content/llama3_lora', '/content/drive/MyDrive/test_G_save/llama3_lora/')

In [ ]:
#import shutil
#from google.colab import files
#files.download('/content/model/model.pth')

In [ ]:
!test -e /content/model/model.pth && echo "File exists" || echo "File does not exist"
!stat /content/model/model.pth
!cp /content/model/model.pth /content/drive/MyDrive/test_G_save/model/

In [ ]:
#import shutil
#from google.colab import files

# 壓縮模型文件夾
#shutil.make_archive('model', 'zip', './model')

# 下載壓縮文件
#files.download('model.zip')


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
# 將壓縮文件移動到Google Drive
#!cp model.zip /content/drive/MyDrive/llm/model.zip
#!cp model.zip /content/drive/MyDrive/llm/

量化


In [ ]:
!git clone https://github.com/ggerganov/llama.cpp.git
%cd llama.cpp
!pip install -r requirements.txt

In [ ]:
!mkdir build
%cd build
!cmake ..
!make

In [ ]:
# 回到主目錄
%cd /content/
# 轉換 .pth 文件為 .gguf 格式
# 假設有一個轉換腳本名為 `convert.py`
# 你需要根據 `llama.cpp` 提供的具體轉換工具和命令來運行
#!python llama.cpp/convert-hf-to-gguf.py --input model.pth --output model.gguf
import requests
from huggingface_hub import HfApi, hf_hub_download

os.environ["HUGGINGFACE_TOKEN"] = "our  HUGGINGFACE_TOKEN"



import os
import shutil

# Set the access token as an environment variable
os.environ["HUGGINGFACE_HUB_CACHE"] = "/content/cache"
os.environ["HUGGINGFACE_TOKEN"] = "your  HUGGINGFACE_TOKEN"

# Initialize the HfApi object (no need to set the token explicitly)
hf_api = HfApi()

# File information
repo_id = "taide/TAIDE-LX-7B"
filename = "config.json"

# Download the file
file_path = hf_hub_download(repo_id=repo_id, filename=filename, use_auth_token=True)

# Copy the file to the desired location
save_path = "/content/model/config.json"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
shutil.copy(file_path, save_path)

print(f"File downloaded and saved to {save_path}")

!python llama.cpp/convert-hf-to-gguf.py   model --outfile model.gguf


In [ ]:
!pip install llama-cpp-python gradio

In [ ]:
import gradio as gr
from llama_cpp import Llama

# Load the model
model_path = "/content/model.gguf"  # Replace with your actual path
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model file not found at: {model_path}")

# Load the model (handling potential errors)
try:
    llm = Llama(model_path=model_path, n_ctx=512)
except Exception as e:
    print(f"Error loading model: {e}")
    # Consider adding more detailed error handling or logging here

def generate_text(prompt):
    output = llm(prompt, max_tokens=32, stop=["\n\n"])  # Customize generation parameters
    return output['choices'][0]['text']

# Create the Gradio interface
demo = gr.Interface(
    fn=generate_text,
    inputs=gr.Textbox(lines=5, label="每天只睡三小時會怎樣"),
    outputs=gr.Textbox(label="Generated Text")
)

demo.launch()

In [ ]:
import gradio as gr

def chatbot_response(input_text):
    inputs = tokenizer.encode(input_text, return_tensors='pt')
    outputs = model.generate(inputs, max_length=100, num_return_sequences=1)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# 建立 Gradio 界面
iface = gr.Interface(fn=chatbot_response, inputs="text", outputs="text")
iface.launch()
